<a href="https://colab.research.google.com/github/tiendk195/hairChangeBySide/blob/master/hairchangebyside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---

🌟 **Ứng dụng thay đổi mẫu tóc với HairFastGAN**

🔗 **[Repository GitHub](https://github.com/AIRI-Institute/HairFastGAN)**

Ứng dụng này cho phép bạn thay đổi mẫu tóc dựa trên model HairFastGAN, giúp bạn có thể tự do tùy chỉnh mẫu tóc và màu tóc phù hợp với khuôn mặt của bạn.

Trước khi bắt đầu, hãy sao chép Colab về driver của bạn để có thể tự do chỉnh sửa:

```
File -> Save a copy to Drive
```

Để bắt đầu, bạn có thể tải lên các hình ảnh của mình cho khuôn mặt, mẫu tóc, và màu tóc. Sau đó, sử dụng ứng dụng để thực hiện chuyển đổi và xem kết quả.

**Lưu ý:** Vui lòng kiểm tra các hướng dẫn trong notebook để đảm bảo rằng bạn đã tuân thủ đúng quy trình và cài đặt cần thiết trước khi sử dụng ứng dụng.

🎨 **Hướng dẫn:**

1. Sao chép Colab về driver của bạn.
2. Tải lên các hình ảnh của bạn cho khuôn mặt, mẫu tóc, và màu tóc.
3. Chạy các ô mã để thực hiện chuyển đổi.
4. Xem kết quả và tùy chỉnh theo ý muốn.



In [ ]:
# @title 1. 📄 **Sao chép dự án từ github repo**
# @markdown 👈Nhấn nút này để clone (~1 phút)
# Clone git repo
!git clone https://github.com/AIRI-Institute/HairFastGAN
%cd HairFastGAN

In [ ]:
# @title 2. ⚙️ **Tải và cài đặt Ninja package**
# @markdown 👈Nhấn nút này để setup (~1 phút)
# Install ninja - small build system to run C++, C
# Install Ninja - small build system https://github.com/ninja-build/ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
# @title 3. ⚙️ **Tải các gói thư viện cần thiết**
# @markdown 👈Nhấn nút này để install (~5 phút)
# Install lib
!pip install pyngrok torchvision streamlit pillow==10.0.0 face_alignment dill==0.2.7.1 addict fpie git+https://github.com/openai/CLIP.git -q


In [ ]:
# @title 4. 📄 **Sao chép model**
# @markdown 👈Nhấn nút này để clone (~3 phút)
# Download pretrain
!git clone https://huggingface.co/AIRI-Institute/HairFastGAN
!cd HairFastGAN && git lfs pull && cd ..
!mv HairFastGAN/pretrained_models pretrained_models
!mv HairFastGAN/input input
!rm -rf HairFastGAN

In [ ]:
# @title 5. ✍ **Khai báo các thư viện**
# @markdown 👈Nhấn nút này để install (~1 phút)
# Khai báo các thư viện
from pathlib import Path
from hair_swap import HairFast, get_parser

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torchvision.transforms as T
import torch

In [ ]:
# @title 6. 🕐 **Khởi tạo model để test**
# @markdown 👈Nhấn nút này để install (~1 phút)
# Khởi tạo model để test
model_args = get_parser()
hair_fast = HairFast(model_args.parse_args([]))

Loading StyleGAN2 from checkpoint: pretrained_models/StyleGAN/ffhq.pt
Loading e4e over the pSp framework from checkpoint: pretrained_models/encoder4editing/e4e_ffhq_encode.pt
Network [SPADEGenerator] was created. Total number of parameters: 266.9 million. To see the architecture, do print(network).


In [ ]:
# @title 7. ⚙️ **Code chức năng**
# @markdown 👈Nhấn nút này để install (~1 phút)
# Display function
def display_images(images=None, **kwargs):
    is_titles = images is None
    images = images or kwargs

    grid = gridspec.GridSpec(1, len(images))
    fig = plt.figure(figsize=(20, 10))

    for i, item in enumerate(images.items() if is_titles else images):
        title, img = item if is_titles else (None, item)

        img = T.functional.to_pil_image(img) if isinstance(img, torch.Tensor) else img
        img = Image.open(img) if isinstance(img, str | Path) else img

        ax = fig.add_subplot(1, len(images), i+1)
        ax.imshow(img)
        if title:
            ax.set_title(title, fontsize=20)
        ax.axis('off')

    plt.show()

In [ ]:
# @title 8. 🔼 **Upload ảnh của bạn**
# @markdown 👈Nhấn nút này để upload (~1 phút)

# @markdown Ảnh của bạn sau khi tải xong sẽ được lưu vào folder 'HairFastGAN/input'
from ipywidgets import FileUpload
import os

input_dir = '/content/HairFastGAN/input'
if not os.path.exists(input_dir):
    os.makedirs(input_dir)

def handle_upload(change):
    uploaded_file = list(upload_button.value.keys())[0]
    content = upload_button.value[uploaded_file]['content']
    with open(f'{input_dir}/{uploaded_file}', 'wb') as f:
        f.write(content)
    print(f"Đã lưu file {uploaded_file} vào thư mục {input_dir}")

upload_button = FileUpload(description="Tải ảnh lên")
upload_button.observe(handle_upload, names='value')
display(upload_button)


In [ ]:
# @title 9. ✅ **Custom hình ảnh kết quả**
# @markdown 👈Nhấn nút này để install (~1 phút)
input_dir = Path("/content/HairFastGAN/input")

# @markdown Thay các biến 'face_path/shape_path/color_path thành các ảnh bạn mong muốn'
face_path = input_dir / '8.png'
shape_path = input_dir / '1.png'
color_path = input_dir / '2.png'

final_image, face_align, shape_align, color_align = hair_fast.swap(face_path,shape_path,color_path, align=True)

%matplotlib inline
display_images(final_result=final_image, input_face=face_align, hair_shape=shape_align, hair_color=color_align)

**Advanced**

Khởi chạy app trên nền tảng streamlit

In [ ]:
# @title 10. ⚙️ **Download streamlit runtime**
# @markdown 👈Nhấn nút này để install (~3 phút)
# Run app.py by streamlit
!streamlit run app.py&>/dev/null&
!pgrep streamlit

In [ ]:
# @title 11. ⚙️ **Set up ngrok server authtoken**
# @markdown 👈Nhấn nút này để install (~1 phút)

# @markdown Truy cập vào https://dashboard.ngrok.com tạo account sau đó get được token
# Expose by grok
!ngrok authtoken <>

In [ ]:
# @title 12. ⚙️ **Chạy ngrok server**
# @markdown 👈Nhấn nút này để install (~1 phút)

# @markdown Sau khi chạy truy cập vào mục endpoint của ngrok sẽ có url dẫn đến website, nếu không render được thì do đầy tài nguyên hãy reset lại
!ngrok http 8501